In [99]:
import pandas as pd
import numpy as np

# Get to know the data

In [105]:
df_source = pd.read_csv("germany_housing_data_14.07.2020.csv", sep=",")

# analzyse sparse columns (= many NaNs/Null-Values)
nulls:pd.Series =df_source.isnull().sum(axis = 0).sort_values(ascending = False)
df_null_info:pd.DataFrame = pd.DataFrame(data={
    'nulls_amount': nulls,
    'nulls_percentage': nulls.apply(lambda row: round((row/df_source.shape[0])*100,2))
})
df_null_info

,nulls_amount,nulls_percentage
Energy_consumption,8119,76.94
Year_renovated,5203,49.31
Usable_area,4984,47.23
Energy_efficiency_class,4819,45.67
Bedrooms,3674,34.82
Free_of_Relation,3569,33.82
Energy_certificate_type,3526,33.42
Furnishing_quality,2726,25.83
Floors,2664,25.25
Garages,1960,18.57


In [106]:
df_source.describe(include="all")

,Unnamed: 0,Price,Type,Living_space,Lot,Usable_area,Free_of_Relation,Rooms,Bedrooms,Bathrooms,...,Energy_source,Energy_certificate,Energy_certificate_type,Energy_consumption,Energy_efficiency_class,State,City,Place,Garages,Garagetype
count,10552.000000,1.055200e+04,10150,10552.000000,10552.000000,5568.000000,6983,10552.000000,6878.000000,8751.000000,...,9325,9797,7026,2433.000000,5733,10551,10551,10262,8592.000000,8592
unique,NaN,NaN,11,NaN,NaN,NaN,705,NaN,NaN,NaN,...,104,3,2,NaN,9,16,534,4762,NaN,7
top,NaN,NaN,Mid-terrace house,NaN,NaN,NaN,nach Absprache,NaN,NaN,NaN,...,Gas,available,demand certificate,NaN,D,Nordrhein-Westfalen,Hannover (Kreis),Innenstadt,NaN,Garage
freq,NaN,NaN,4324,NaN,NaN,NaN,1383,NaN,NaN,NaN,...,4528,6987,4143,NaN,970,1660,107,32,NaN,4412
mean,5275.500000,5.566851e+05,NaN,216.721008,1491.659004,134.300424,NaN,7.388978,4.169817,2.308993,...,NaN,NaN,NaN,117.663111,NaN,NaN,NaN,NaN,2.698673,NaN
std,3046.244354,6.087410e+05,NaN,172.421321,8582.361675,188.814089,NaN,5.378126,2.577169,1.742330,...,NaN,NaN,NaN,54.023207,NaN,NaN,NaN,NaN,3.195068,NaN
min,0.000000,0.000000e+00,NaN,0.000000,0.000000,0.000000,NaN,1.000000,0.000000,0.000000,...,NaN,NaN,NaN,5.100000,NaN,NaN,NaN,NaN,1.000000,NaN
25%,2637.750000,2.500000e+05,NaN,130.000000,370.000000,48.000000,NaN,5.000000,3.000000,1.000000,...,NaN,NaN,NaN,83.170000,NaN,NaN,NaN,NaN,1.000000,NaN
50%,5275.500000,4.052150e+05,NaN,176.775000,656.500000,80.000000,NaN,6.000000,4.000000,2.000000,...,NaN,NaN,NaN,112.700000,NaN,NaN,NaN,NaN,2.000000,NaN
75%,7913.250000,6.550000e+05,NaN,250.000000,1047.000000,150.250000,NaN,8.000000,5.000000,3.000000,...,NaN,NaN,NaN,146.000000,NaN,NaN,NaN,NaN,3.000000,NaN


# Pre-Processing

### Data Reduction

In [112]:
# Drop all columns with more than 26% null values
drop_col_percentage_treshold=26

cols_to_drop = list(df_null_info[(df_null_info['nulls_percentage'] > drop_col_percentage_treshold)].index.values) 
print(f'Drop following columns: {cols_to_drop}')
df_source.drop(columns=cols_to_drop, inplace=True)

df_state_city = df_source[["State", "City"]]

Drop following columns: []


In [119]:
df_state_city.groupby('State').nunique()

,City
State,
Baden-Württemberg,44
Bayern,96
Berlin,57
Brandenburg,18
Bremen,29
Hamburg,30
Hessen,26
Mecklenburg-Vorpommern,22
Niedersachsen,47


### Data Cleaning (handle noisy & missing data)

In [108]:
nulls:pd.Series =df_source.isnull().sum(axis = 0).sort_values(ascending = False)
df_null_info:pd.DataFrame = pd.DataFrame(data={
    'nulls_amount': nulls,
    'nulls_percentage': nulls.apply(lambda row: round((row/df_source.shape[0])*100,2))
})
df_null_info

,nulls_amount,nulls_percentage
Furnishing_quality,2726,25.83
Floors,2664,25.25
Garages,1960,18.57
Garagetype,1960,18.57
Bathrooms,1801,17.07
Energy_source,1227,11.63
Energy_certificate,755,7.16
Year_built,694,6.58
Heating,584,5.53
Type,402,3.81
